In [1]:
### https://github.com/allenai/longformer/blob/master/scripts/convert_model_to_long.ipynb

In [2]:
### This notebook needs to be converted to python file and run with deepspeed filename.py
### DeepSpeed needs to have https://github.com/microsoft/DeepSpeed/pull/5780 integrated into deepspeed/ops/op_builder/builder.py
### LongformerSelfAttention forward have a line that needs to be changed
### remove_from_windowed_attention_mask = (attention_mask != 0)[:, :, None, None]
### have incorrect shape because the thing that is passed in is [a, 1, 1, b]
### so need to change it to 
### remove_from_windowed_attention_mask = (attention_mask != 0)[:, 0, 0, :, None, None]
### Another problem occurs where the code in the same function
### attn_probs = torch.masked_fill(attn_probs, is_index_masked[:, :, None, None], 0.0)
### does not check if is_index_masked is None
### So this needs to be changed to
### if is_index_masked:
###     attn_probs = torch.masked_fill(attn_probs, is_index_masked[:, :, None, None], 0.0)

In [3]:
import logging
import os
import math
import copy
import torch
from dataclasses import dataclass, field
from transformers import BertForMaskedLM, RobertaTokenizerFast, TextDataset, DataCollatorForLanguageModeling, Trainer
from transformers import TrainingArguments, HfArgumentParser, AutoModelForMaskedLM, BertTokenizerFast
from transformers.models.longformer.modeling_longformer import LongformerSelfAttention
from transformers import AutoTokenizer, AutoModel

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [4]:
class LegalBertLongSelfAttention(LongformerSelfAttention):
    def forward(
        self,
        hidden_states,
        attention_mask=None,
        head_mask=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        past_key_value=None,
        output_attentions=False,
        **kwargs,
    ):
        return super().forward(hidden_states, attention_mask=attention_mask, output_attentions=output_attentions)


class LegalBertLongForMaskedLM(BertForMaskedLM):
    def __init__(self, config):
        super().__init__(config)
        for i, layer in enumerate(self.bert.encoder.layer):
            # replace the `modeling_bert.BertSelfAttention` object with `LongformerSelfAttention`
            layer.attention.self = LegalBertLongSelfAttention(config, layer_id=i)

In [5]:
def copy_proj_layers(model):
    for i, layer in enumerate(model.bert.encoder.layer):
        layer.attention.self.query_global = copy.deepcopy(layer.attention.self.query)
        layer.attention.self.key_global = copy.deepcopy(layer.attention.self.key)
        layer.attention.self.value_global = copy.deepcopy(layer.attention.self.value)
    return model

In [6]:
@dataclass
class ModelArgs:
    attention_window: int = field(default=512, metadata={"help": "Size of attention window"})
    max_pos: int = field(default=4096, metadata={"help": "Maximum position"})

model_args = ModelArgs()

model_hidden_size = 512
train_batch_size = 1

training_args = TrainingArguments(
    output_dir = "tmp",
    max_steps = 3000,
    logging_steps = 500,
    save_steps = 500,
    fp16=True,
    per_device_train_batch_size=1,
    do_train = True,
    do_eval = True,
    # deepspeed="ds_config.json",
)
training_args.val_datapath = 'wikitext-103-raw/wiki.valid.raw'
training_args.train_datapath = 'wikitext-103-raw/wiki.train.raw'

model_path = f'{training_args.output_dir}/legalbert-{model_args.max_pos}'
if not os.path.exists(model_path):
    os.makedirs(model_path)

In [7]:
def pretrain_and_evaluate(args, model, tokenizer, eval_only, model_path):
    val_dataset = TextDataset(tokenizer=tokenizer,
                              file_path=args.val_datapath,
                              block_size=tokenizer.model_max_length)
    if eval_only:
        train_dataset = val_dataset
    else:
        logger.info(f'Loading and tokenizing training data is usually slow: {args.train_datapath}')
        train_dataset = TextDataset(tokenizer=tokenizer,
                                    file_path=args.train_datapath,
                                    block_size=tokenizer.model_max_length)

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)
    trainer = Trainer(model=model, args=args, data_collator=data_collator,
                      train_dataset=train_dataset, eval_dataset=val_dataset) # , prediction_loss_only=True,)

    """
    ### CANT DO EVALUATE HERE. IT MESSES UP WITH SETTING UP DEEPSPEED FOR TRAINING ### 
    eval_loss = trainer.evaluate()
    eval_loss = eval_loss['eval_loss']
    logger.info(f'Initial eval bpc: {eval_loss/math.log(2)}')
    """
    
    if not eval_only:
        trainer.train()
        trainer.save_model()

        eval_loss = trainer.evaluate()
        eval_loss = eval_loss['eval_loss']
        logger.info(f'Eval bpc after pretraining: {eval_loss/math.log(2)}')

    return trainer

In [8]:
logger.info(f'Loading the model from {model_path}')
tokenizer = AutoTokenizer.from_pretrained(model_path, model_max_length=4096)
model = LegalBertLongForMaskedLM.from_pretrained(model_path)

INFO:__main__:Loading the model from tmp/roberta-base-4096
/home/yan_xu_uk_qbe_com/scc_yan/virtual-env/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
tmp_save_path = "tmp_save"
if not os.path.exists(tmp_save_path):
    os.mkdir(tmp_save_path)

In [10]:
from deepspeed.utils.zero_to_fp32 import get_fp32_state_dict_from_zero_checkpoint

state_dict = get_fp32_state_dict_from_zero_checkpoint(tmp_save_path) # already on cpu
print({k: type(v) for k, v in state_dict.items()})

[2024-08-19 14:15:46,406] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/opt/conda/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.4
 [WARNING]  using untested triton version (3.0.0), only 1.0.0 is known to be compatible
Processing zero checkpoint 'tmp_save/global_step3'


/home/yan_xu_uk_qbe_com/scc_yan/virtual-env/lib/python3.10/site-packages/deepspeed/runtime/zero/linear.py:49: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, weight, bias=None):
/home/yan_xu_uk_qbe_com/scc_yan/virtual-env/lib/python3.10/site-packages/deepspeed/runtime/zero/linear.py:67: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/home/yan_xu_uk_qbe_com/scc_yan/virtual-env/lib/python3.10/site-packages/deepspeed/utils/zero_to_fp32.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#un

Detected checkpoint of type zero stage 3, world_size: 8
Parsing checkpoint created by deepspeed==0.14.4


/home/yan_xu_uk_qbe_com/scc_yan/virtual-env/lib/python3.10/site-packages/deepspeed/utils/zero_to_fp32.py:98: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch

Reconstructed Trainable fp32 state dict with 274 params 133529658 elements
{'bert.embeddings.word_embeddings.weight': <class 'torch.Tensor'>, 'bert.embeddings.position_embeddings.weight': <class 'torch.Tensor'>, 'bert.embeddings.token_type_embeddings.weight': <class 'torch.Tensor'>, 'bert.embeddings.LayerNorm.weight': <class 'torch.Tensor'>, 'bert.embeddings.LayerNorm.bias': <class 'torch.Tensor'>, 'bert.encoder.layer.0.attention.self.query.weight': <class 'torch.Tensor'>, 'bert.encoder.layer.0.attention.self.query.bias': <class 'torch.Tensor'>, 'bert.encoder.layer.0.attention.self.key.weight': <class 'torch.Tensor'>, 'bert.encoder.layer.0.attention.self.key.bias': <class 'torch.Tensor'>, 'bert.encoder.layer.0.attention.self.value.weight': <class 'torch.Tensor'>, 'bert.encoder.layer.0.attention.self.value.bias': <class 'torch.Tensor'>, 'bert.encoder.layer.0.attention.self.query_global.weight': <class 'torch.Tensor'>, 'bert.encoder.layer.0.attention.self.query_global.bias': <class 'torc

In [11]:
from deepspeed.utils.zero_to_fp32 import load_state_dict_from_zero_checkpoint
model = LegalBertLongForMaskedLM.from_pretrained(model_path)
model = load_state_dict_from_zero_checkpoint(model, tmp_save_path)
# loaded_model.load_state_dict(state_dict, strict=False)
# loaded_model.load_state_dict(state_dict)
print(model)

[2024-08-19 14:15:48,488] [INFO] [zero_to_fp32.py:570:load_state_dict_from_zero_checkpoint] Extracting fp32 weights
Processing zero checkpoint 'tmp_save/global_step3'
Detected checkpoint of type zero stage 3, world_size: 8
Parsing checkpoint created by deepspeed==0.14.4
Reconstructed Trainable fp32 state dict with 274 params 133529658 elements
[2024-08-19 14:15:49,196] [INFO] [zero_to_fp32.py:573:load_state_dict_from_zero_checkpoint] Overwriting model with fp32 weights
LegalBertLongForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(4098, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): LegalBertLong

In [12]:
model_path_global = "model_path_global"
if not os.path.exists(model_path_global):
    os.mkdir(model_path_global)
logger.info(f'Copying local projection layers into global projection layers ... ')
model = copy_proj_layers(model)
logger.info(f'Saving model to {model_path_global}')
model.save_pretrained(model_path_global)

INFO:__main__:Copying local projection layers into global projection layers ... 
INFO:__main__:Saving model to model_path_global


In [13]:
logger.info(f'Loading the model from {model_path_global}')
### tokenizer = RobertaTokenizerFast.from_pretrained(model_path_global)
model = LegalBertLongForMaskedLM.from_pretrained(model_path_global)

INFO:__main__:Loading the model from model_path_global


In [14]:
model.bert.embeddings.word_embeddings.weight

Parameter containing:
tensor([[ 0.0126, -0.0295, -0.0126,  ..., -0.0294, -0.0417, -0.0240],
        [-0.0580,  0.0049,  0.0112,  ..., -0.0799, -0.0648, -0.0321],
        [-0.0230,  0.0208, -0.0591,  ..., -0.0287, -0.0791, -0.0778],
        ...,
        [-0.0188,  0.0665,  0.0168,  ...,  0.0267, -0.0039, -0.0428],
        [-0.0558,  0.0061,  0.0137,  ..., -0.0819,  0.0236,  0.0089],
        [-0.0174,  0.0405,  0.0303,  ..., -0.0673, -0.0832,  0.0496]],
       requires_grad=True)